# Spark - Decision Tree, Random Forest and GBT Classifier

**Imports**

In [59]:
import findspark
findspark.init('/home/sedat/spark-3.3.2-bin-hadoop3')
from pyspark.sql import SparkSession
from pyspark.ml import pipeline
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MultilabelClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StringIndexer

**Start spark session and read csv**

In [5]:
spark = SparkSession.builder.appName('my_trees').getOrCreate()

In [8]:
df = spark.read.csv('College.csv', inferSchema=True, header=True)

In [10]:
df.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [11]:
df.count()

777

In [15]:
df.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

**Vector Assembler**

In [16]:
df.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [19]:
assembler_col = df.columns[2:]
assembler_col

['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [20]:
assembler = VectorAssembler(inputCols=assembler_col,
                           outputCol='features')

In [23]:
output = assembler.transform(df)

**String Indexer for Private column**

In [28]:
indexer = StringIndexer(inputCol='Private', outputCol='Private_Index')

In [29]:
output_fixed = indexer.fit(output).transform(output)

In [30]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- Private_Index: double (nullable = false)



**Final data and split them**

In [39]:
final_data = output_fixed.select('Private_Index', 'features')

In [40]:
final_data.show()

+-------------+--------------------+
|Private_Index|            features|
+-------------+--------------------+
|          0.0|[1660.0,1232.0,72...|
|          0.0|[2186.0,1924.0,51...|
|          0.0|[1428.0,1097.0,33...|
|          0.0|[417.0,349.0,137....|
|          0.0|[193.0,146.0,55.0...|
|          0.0|[587.0,479.0,158....|
|          0.0|[353.0,340.0,103....|
|          0.0|[1899.0,1720.0,48...|
|          0.0|[1038.0,839.0,227...|
|          0.0|[582.0,498.0,172....|
|          0.0|[1732.0,1425.0,47...|
|          0.0|[2652.0,1900.0,48...|
|          0.0|[1179.0,780.0,290...|
|          0.0|[1267.0,1080.0,38...|
|          0.0|[494.0,313.0,157....|
|          0.0|[1420.0,1093.0,22...|
|          0.0|[4302.0,992.0,418...|
|          0.0|[1216.0,908.0,423...|
|          0.0|[1130.0,704.0,322...|
|          1.0|[3540.0,2001.0,10...|
+-------------+--------------------+
only showing top 20 rows



In [41]:
train_df, test_df = final_data.randomSplit([0.7, 0.3])

**Models and Fits**

In [45]:
dtc = DecisionTreeClassifier(labelCol='Private_Index')
rfc = RandomForestClassifier(labelCol='Private_Index')
gbt = GBTClassifier(labelCol='Private_Index')

In [47]:
dtc_model = dtc.fit(train_df)
rfc_model = rfc.fit(train_df)
gbt_model = gbt.fit(train_df)

**Test Predictions**

In [48]:
dtc_preds = dtc_model.transform(test_df)
rfc_preds = rfc_model.transform(test_df)
gbt_preds = gbt_model.transform(test_df)

**Evaluations with Binary Classifier (roc_auc)**

In [50]:
my_binary_eval = BinaryClassificationEvaluator(labelCol='Private_Index')

In [58]:
print('DTC roc_auc score:', my_binary_eval.evaluate(dtc_preds))
print('RFC roc_auc score:', my_binary_eval.evaluate(rfc_preds))
print('GBT roc_auc score:', my_binary_eval.evaluate(gbt_preds))

DTC roc_auc score: 0.9203781512605043
RFC roc_auc score: 0.9782352941176468
GBT roc_auc score: 0.9673109243697477


**Evaluations with Multiple Classifier (accuracy, recal, precision, f1)**

In [70]:
my_acc_eval = MulticlassClassificationEvaluator(labelCol='Private_Index', metricName='f1')

In [72]:
print('DTC f1 score:', my_acc_eval.evaluate(dtc_preds))
print('RFC f1 score:', my_acc_eval.evaluate(rfc_preds))
print('GBT f1 score:', my_acc_eval.evaluate(gbt_preds))

DTC f1 score: 0.9226304455687399
RFC f1 score: 0.9301962489496332
GBT f1 score: 0.9265559531455485
